In [1]:
import torch

In [2]:
torch.__version__

'1.4.0'

In [3]:
rotation = torch.zeros(40, 3, 3, requires_grad=True)
translation = torch.zeros(40, 3, requires_grad=True)
focal_length = torch.zeros(1, requires_grad=True)
distrotion =  torch.zeros(1, requires_grad=True)
camera_px = torch.tensor(460.0, requires_grad=False)
camera_py = torch.tensor(608.0, requires_grad=False)

In [4]:
collect_point2d = []
views_list = []
point3d_list = []
views_count = 0
point3d_count = 0
uv_count = 0
with open('penguin_feature_matching.txt','r') as f:
    views_count, point3d_count, uv_count = f.readline().strip().split(' ')
    collect_point2d = []
    for _ in range(int(uv_count)):
        view, p3d, u,v = list(filter(None,f.readline().strip().split(' ')))
        collect_point2d.append([float(u),float(v)])
        views_list.append(int(view))
        point3d_list.append(int(p3d))

In [5]:
point3d = torch.rand(int(point3d_count), 3, requires_grad=True)
point2d = torch.tensor(collect_point2d,requires_grad=False)
point3d_index = torch.tensor(point3d_list,dtype=torch.long,requires_grad=False)
extrinsic_index = torch.tensor(views_list,dtype=torch.long,requires_grad=False)

In [6]:
intrinsic = torch.zeros(3,3, requires_grad=False)
intrinsic[0,0] = focal_length
intrinsic[1,1] = focal_length
intrinsic[0,1] = distrotion
intrinsic[0,2] = camera_px
intrinsic[1,2] = camera_py
intrinsic[2,2] = 1.0

In [7]:
extrinsic_pad = torch.zeros(int(uv_count),4,4, requires_grad=False)
extrinsic_pad[:,:,3] = 1
extrinsic_pad[:,:3,:3] = rotation[extrinsic_index]
extrinsic_pad[:,3,:3] = translation[extrinsic_index]

In [8]:
point3d_pad = torch.ones(int(uv_count),4,1, requires_grad=False)
point3d_pad[:,:3,0] = point3d[point3d_index]

In [15]:
#optimizer = torch.optim.Adam([rotation, translation, focal_length,distrotion,point3d], lr=0.01)
# use per parameter learning rate
optimizer = torch.optim.Adam([
    {'params':focal_length, 'lr':1},
    {'params':rotation},
    {'params':translation},
    {'params':distrotion},
    {'params':point3d},
], lr=0.01) 

In [21]:
for i in range(10000):
    optimizer.zero_grad()
    # Do projection
    extrinsic_pad_point = torch.matmul(extrinsic_pad,point3d_pad)
    extrinsic_pad_point = extrinsic_pad_point[:,:3,:]
    projected_point = torch.matmul(intrinsic,extrinsic_pad_point[:,:3,:])
    projected_point = projected_point[:,:,0]
    uv_map = torch.empty(int(uv_count),2)
    uv_map[:,0] = projected_point[:,0] / projected_point[:,2]
    uv_map[:,1] = projected_point[:,1] / projected_point[:,2]
    # compare after projection with point2d
    uv_difference = torch.pow(point2d - uv_map, 2)
    total_loss = uv_difference.sum()
    total_loss.backward(retain_graph=True)
    optimizer.step()
    if i % 100 == 0:
        print("EPOCH %d - loss %f" % (i,total_loss.item()))

EPOCH 0 - loss -2504700.500000
EPOCH 100 - loss -2504700.500000
EPOCH 200 - loss -2504700.500000
EPOCH 300 - loss -2504700.500000
EPOCH 400 - loss -2504700.500000
EPOCH 500 - loss -2504700.500000
EPOCH 600 - loss -2504700.500000
EPOCH 700 - loss -2504700.500000
EPOCH 800 - loss -2504700.500000
EPOCH 900 - loss -2504700.500000


In [38]:
x = torch.tensor([5.0],requires_grad=True)
y = torch.tensor([5.0],requires_grad=True)
opt = torch.optim.Adam([x,y],lr=0.1) 
for i in range(1000):
    opt.zero_grad()
    total_loss = torch.pow((x-3)*(y-4),2)
    total_loss.backward(retain_graph=True)
    opt.step()
    if i % 10 == 0:
        print("EPOCH %d - loss %f - x = %f, y = %f" % (i,total_loss.item(),x,y))

EPOCH 0 - loss 4.000000 - x = 4.900000, y = 4.900000
EPOCH 10 - loss 0.025425 - x = 4.128277, y = 4.074530
EPOCH 20 - loss 0.044304 - x = 3.810846, y = 3.730544
EPOCH 30 - loss 0.041227 - x = 3.622583, y = 3.684890
EPOCH 40 - loss 0.016944 - x = 3.471735, y = 3.737942
EPOCH 50 - loss 0.006064 - x = 3.366236, y = 3.797495
EPOCH 60 - loss 0.002496 - x = 3.301455, y = 3.840520
EPOCH 70 - loss 0.001270 - x = 3.263405, y = 3.868524
EPOCH 80 - loss 0.000771 - x = 3.240524, y = 3.886937
EPOCH 90 - loss 0.000529 - x = 3.225819, y = 3.899833
EPOCH 100 - loss 0.000390 - x = 3.215551, y = 3.909602
EPOCH 110 - loss 0.000301 - x = 3.207819, y = 3.917529
EPOCH 120 - loss 0.000238 - x = 3.201656, y = 3.924288
EPOCH 130 - loss 0.000192 - x = 3.196561, y = 3.930239
EPOCH 140 - loss 0.000156 - x = 3.192254, y = 3.935582
EPOCH 150 - loss 0.000129 - x = 3.188565, y = 3.940438
EPOCH 160 - loss 0.000106 - x = 3.185382, y = 3.944884
EPOCH 170 - loss 0.000088 - x = 3.182623, y = 3.948979
EPOCH 180 - loss 0.00